In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_path = "/content/drive/My Drive/INMB_Main/"


In [ ]:
!unzip "/content/drive/My Drive/INMB_Main/INMB.zip" -d "/content/"


Archive:  /content/drive/My Drive/INMB_Main/INMB.zip
replace /content/INMB/Test_INMB/no/1 no.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/INMB/Test_INMB/no/1 no.jpeg  
  inflating: /content/INMB/Test_INMB/no/10 no.jpg  
  inflating: /content/INMB/Test_INMB/no/11 no.jpg  
  inflating: /content/INMB/Test_INMB/no/12 no.jpg  
  inflating: /content/INMB/Test_INMB/no/13 no.jpg  
  inflating: /content/INMB/Test_INMB/no/14 no.jpg  
  inflating: /content/INMB/Test_INMB/no/15 no.jpg  
  inflating: /content/INMB/Test_INMB/no/17 no.jpg  
  inflating: /content/INMB/Test_INMB/no/18 no.jpg  
  inflating: /content/INMB/Test_INMB/no/19 no.jpg  
  inflating: /content/INMB/Test_INMB/no/2 no.jpeg  
  inflating: /content/INMB/Test_INMB/no/20 no.jpg  
  inflating: /content/INMB/Test_INMB/no/21 no.jpg  
  inflating: /content/INMB/Test_INMB/no/22 no.jpg  
  inflating: /content/INMB/Test_INMB/no/23 no.jpg  
  inflating: /content/INMB/Test_INMB/no/24 no.jpg  
  inflating: /content/INMB/Te

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator



# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% for validation
)

# Load datasets
train_data = datagen.flow_from_directory(
    '/content/INMB/Train_INMB/brain_tumor_dataset',
    target_size=(250, 250),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    '/content/INMB/Train_INMB',
    target_size=(250, 250),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 203 images belonging to 2 classes.
Found 50 images belonging to 1 classes.


In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Initialize ImageDataGenerator with validation split
# datagen = ImageDataGenerator(validation_split=0.2)  # 20% for validation

# # Load training data (80% of the dataset)
# train_data = datagen.flow_from_directory(
#     '/content/INMB/Train_INMB',   # Assuming this directory contains all the data you want to split
#     target_size=(250, 250),
#     batch_size=32,
#     class_mode='binary',
#     subset='training'  # Use the 'training' subset of the data
# )

# # Load validation data (20% of the dataset)
# val_data = datagen.flow_from_directory(
#     '/content/INMB/Train_INMB',   # Note: We use the same directory as before
#     target_size=(250, 250),
#     batch_size=32,
#     class_mode='binary',
#     subset='validation'  # Use the 'validation' subset of the data
# )


In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# model = Sequential([
#     Conv2D(64, (3, 3), activation='relu', input_shape=(250, 250, 3)),
#     MaxPooling2D(2, 2),
#     Conv2D(128, (3, 3), activation='relu'),
#     MaxPooling2D(2, 2),
#     Conv2D(256, (3, 3), activation='relu'),
#     MaxPooling2D(2, 2),
#     Flatten(),
#     Dense(512, activation='relu'),
#     Dropout(0.5),
#     Dense(1, activation='sigmoid')
# ])

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# history = model.fit(train_data, validation_data=val_data, epochs=00)


Epoch 1/100
7/7 [==============================] - 4s 597ms/step - loss: 0.6652 - accuracy: 0.6305 - val_loss: 0.7527 - val_accuracy: 0.3000
Epoch 2/100
7/7 [==============================] - 4s 541ms/step - loss: 0.6408 - accuracy: 0.6207 - val_loss: 0.8844 - val_accuracy: 0.1200
Epoch 3/100
7/7 [==============================] - 5s 787ms/step - loss: 0.6460 - accuracy: 0.6158 - val_loss: 0.8216 - val_accuracy: 0.1600
Epoch 4/100
7/7 [==============================] - 4s 530ms/step - loss: 0.6251 - accuracy: 0.6502 - val_loss: 0.9785 - val_accuracy: 0.2600
Epoch 5/100
7/7 [==============================] - 4s 537ms/step - loss: 0.6336 - accuracy: 0.6749 - val_loss: 0.7618 - val_accuracy: 0.4200
Epoch 6/100
7/7 [==============================] - 5s 719ms/step - loss: 0.6160 - accuracy: 0.6897 - val_loss: 0.9046 - val_accuracy: 0.2800
Epoch 7/100
7/7 [==============================] - 4s 515ms/step - loss: 0.6932 - accuracy: 0.5862 - val_loss: 0.6990 - val_accuracy: 0.4200
Epoch 8/100
7

In [ ]:
from tensorflow.keras.applications import VGG16

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(250, 250, 3))


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense


for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification

model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_data, validation_data=val_data, epochs=20)


Epoch 1/20
7/7 [==============================] - 17s 2s/step - loss: 44.3753 - accuracy: 0.5714 - val_loss: 55.6891 - val_accuracy: 0.0000e+00
Epoch 2/20
7/7 [==============================] - 4s 568ms/step - loss: 10.1722 - accuracy: 0.5517 - val_loss: 1.4822e-04 - val_accuracy: 1.0000
Epoch 3/20
7/7 [==============================] - 5s 788ms/step - loss: 2.9474 - accuracy: 0.5764 - val_loss: 6.2918 - val_accuracy: 0.0200
Epoch 4/20
7/7 [==============================] - 4s 565ms/step - loss: 1.3662 - accuracy: 0.7438 - val_loss: 1.0015 - val_accuracy: 0.7800
Epoch 5/20
7/7 [==============================] - 4s 564ms/step - loss: 0.6550 - accuracy: 0.8424 - val_loss: 0.5877 - val_accuracy: 0.8200
Epoch 6/20
7/7 [==============================] - 5s 767ms/step - loss: 0.5956 - accuracy: 0.8325 - val_loss: 1.7408 - val_accuracy: 0.6000
Epoch 7/20
7/7 [==============================] - 4s 599ms/step - loss: 0.4332 - accuracy: 0.9015 - val_loss: 0.4927 - val_accuracy: 0.8800
Epoch 8/20


In [ ]:
from tensorflow.keras.optimizers import Adam

# Unfreeze some layers of the base model
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Use a smaller learning rate
optimizer = Adam(learning_rate=0.00001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Continue training
history_fine_tune = model.fit(train_data, validation_data=val_data, epochs=10)


Epoch 1/10
7/7 [==============================] - 8s 642ms/step - loss: 0.1444 - accuracy: 0.9409 - val_loss: 0.3260 - val_accuracy: 0.8800
Epoch 2/10
7/7 [==============================] - 4s 633ms/step - loss: 0.0782 - accuracy: 0.9754 - val_loss: 0.1281 - val_accuracy: 0.9800
Epoch 3/10
7/7 [==============================] - 5s 733ms/step - loss: 0.0669 - accuracy: 0.9803 - val_loss: 0.2936 - val_accuracy: 0.9400
Epoch 4/10
7/7 [==============================] - 4s 567ms/step - loss: 0.0957 - accuracy: 0.9606 - val_loss: 0.3979 - val_accuracy: 0.9400
Epoch 5/10
7/7 [==============================] - 4s 576ms/step - loss: 0.1181 - accuracy: 0.9655 - val_loss: 0.1079 - val_accuracy: 0.9600
Epoch 6/10
7/7 [==============================] - 5s 772ms/step - loss: 0.0502 - accuracy: 0.9901 - val_loss: 0.1040 - val_accuracy: 0.9600
Epoch 7/10
7/7 [==============================] - 5s 643ms/step - loss: 0.0941 - accuracy: 0.9754 - val_loss: 0.0941 - val_accuracy: 0.9600
Epoch 8/10
7/7 [====

In [ ]:
model.save("fine_tuned_model.h5")


In [ ]:
!pip install flask-ngrok


In [ ]:
from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf

app = Flask(__name__)

# Load the model
model = tf.keras.models.load_model("fine_tuned_model.h5")

@app.route('/predict', methods=['POST'])
def predict():
    # Get the image from the POST request
    img = tf.image.decode_image(request.files.get('image').read())
    img = tf.image.resize(img, [250, 250])
    img = np.expand_dims(img, axis=0)

    # Make prediction
    predictions = model.predict(img)
    prediction = "Tumor" if predictions[0][0] > 0.5 else "No Tumor"

    # Send back the result as JSON
    return jsonify(prediction=prediction)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
# from flask import Flask, request, jsonify
# from flask_ngrok import run_with_ngrok
# import tensorflow as tf

# app = Flask(__name__)
# run_with_ngrok(app)  # Start ngrok when the app runs

# # Load the model
# model = tf.keras.models.load_model("fine_tuned_model.h5")  # Ensure the model path is correct

# @app.route('/predict', methods=['POST'])
# def predict():
#     # Get the image from the POST request
#     img = tf.image.decode_image(request.files.get('image').read())
#     img = tf.image.resize(img, [250, 250])
#     img = tf.expand_dims(img, axis=0)

#     # Make a prediction
#     predictions = model.predict(img)
#     prediction = "Tumor" if predictions[0][0] > 0.5 else "No Tumor"

#     # Return the result as JSON
#     return jsonify(prediction=prediction)

# if __name__ == '__main__':
#     app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/serving.py", line 748, in __init__
    self.server_bind()
  File "/usr/lib/python3.10/http/server.py", line 137, in server_bind
    socketserver.TCPServer.server_bind(self)
  File "/usr/lib/python3.10/socketserver.py", line 466, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 98] Address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-31-1194e37aea7e>", line 26, in <cell line: 25>
    app.run()
  File "/usr/local/lib/python3.10/dist-packages/flask_ngrok.py", line 88, in new_run
    old_run()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1191, in run
    run_simple(t.cast(str, host), port, self, **op

TypeError: ignored

In [ ]:
# from flask import Flask, request, jsonify
# from flask_ngrok import run_with_ngrok
# import tensorflow as tf

# app = Flask(__name__)
# run_with_ngrok(app)  # Start ngrok when the app runs

# # Load the model
# model = tf.keras.models.load_model("fine_tuned_model.h5")  # Ensure the model path is correct

# @app.route('/')
# def index():
#     return "Welcome to the MRI Brain Tumor Classifier. Use the /predict endpoint to classify images."

# @app.route('/predict', methods=['POST'])
# def predict():
#     # Get the image from the POST request
#     img = tf.image.decode_image(request.files.get('image').read())
#     img = tf.image.resize(img, [250, 250])
#     img = tf.expand_dims(img, axis=0)

#     # Make a prediction
#     predictions = model.predict(img)
#     prediction = "Tumor" if predictions[0][0] > 0.5 else "No Tumor"

#     # Return the result as JSON
#     return jsonify(prediction=prediction)

# if __name__ == '__main__':
#     app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/serving.py", line 748, in __init__
    self.server_bind()
  File "/usr/lib/python3.10/http/server.py", line 137, in server_bind
    socketserver.TCPServer.server_bind(self)
  File "/usr/lib/python3.10/socketserver.py", line 466, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 98] Address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-30-d69484d32f1a>", line 30, in <cell line: 29>
    app.run()
  File "/usr/local/lib/python3.10/dist-packages/flask_ngrok.py", line 88, in new_run
    old_run()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1191, in run
    run_simple(t.cast(str, host), port, self, **op

TypeError: ignored

In [ ]:
!ngrok authtoken 2Ur3qSmZpekUuXmU4m0lrP0FVdM_4MWVc9YV77zCGDEwd2pMP


/bin/bash: line 1: ngrok: command not found


In [ ]:

!unzip /content/ngrok-v3-stable-windows-amd64.zip


Archive:  /content/ngrok-v3-stable-windows-amd64.zip
  inflating: ngrok.exe               


In [ ]:
!./ngrok authtoken 2Ur3qSmZpekUuXmU4m0lrP0FVdM_4MWVc9YV77zCGDEwd2pMP


/bin/bash: line 1: ./ngrok: No such file or directory


In [ ]:
get_ipython().system_raw('./ngrok http 5000 &')


In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
"import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.10/json/__init__.py", line 293, in load
    return loads(fp.read(),
  File "/usr/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


In [ ]:
import threading

def run_flask_app():
    app.run()

threading.Thread(target=run_flask_app).start()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
